In [ ]:
# Import system packages
using Printf
using DelimitedFiles
using Plots
using LaTeXStrings

In [ ]:
# Import AGNI
ROOT_DIR = abspath(pwd(),"../")
using AGNI

# Disable logging from AGNI module
AGNI.setup_logging("",1)

In [ ]:
R_earth = 6.371e6
M_earth = 5.972e24
S_earth = 1362.0

In [ ]:
# Configuration options
instellation    = 5.01*S_earth
mass            = 2.14*M_earth
radius          = 1.273*R_earth
gravity         = 6.67e-11 * mass / radius^2
p_surf          = 1e4
tmp_surf        = 3000.0
s0_fact         = 0.25
zenith_degrees  = 54.74
albedo_b        = 0.0
nlev_centre     = 36
p_top           = 1e-5      # bar

x_H2O = 0.01
mole_fractions  = Dict([
                        ("H2O", x_H2O),
                        ("H2" , 1-x_H2O)
                    ])
thermo          = true
real_gas        = true

phys.ENABLE_AQUA = true

albedo_data   = joinpath(ROOT_DIR,"res/surface_albedos/lunar_marebasalt.dat")
spectral_file = joinpath(ROOT_DIR,"res/spectral_files/Dayspring/48/Dayspring.sf")
star_file     = joinpath(ROOT_DIR,"res/stellar_spectra/sun.txt")
output_dir    = joinpath(ROOT_DIR,"out/")

In [ ]:
# Create output directory
rm(output_dir,force=true,recursive=true)
mkdir(output_dir);

In [ ]:
# Setup atmosphere
atmos = atmosphere.Atmos_t()
atmosphere.setup!(atmos, ROOT_DIR, output_dir,
                        spectral_file,
                        instellation, s0_fact, albedo_b, zenith_degrees,
                        tmp_surf,
                        gravity, radius,
                        nlev_centre, p_surf, p_top,
                        mole_fractions, "",

                        flux_int = 0.0,
                        flag_gcontinuum=true,
                        flag_rayleigh=true,
                        real_gas=real_gas,
                        thermo_functions=thermo,
                        surface_material=albedo_data
                    )
atmosphere.allocate!(atmos, star_file)

In [ ]:
x_arr = collect(Float64, range(0.95, 0.05, length=4))
nsamps = length(x_arr)
display(nsamps)
display(x_arr)

In [ ]:
atm_con_ide = atmosphere.Atmos_t[] # convective, ideal
atm_con_rea = atmosphere.Atmos_t[] # convective, real
@printf("Running convective model for 2*%d samples... \n", nsamps)
for i in 1:nsamps
    @printf("    sample %d/%d \n",i,nsamps)

    # update planet
    x_H2O = x_arr[i]
    println("    x_H2O = $x_H2O")
    fill!(atmos.gas_vmr["H2O"], x_H2O)
    fill!(atmos.gas_vmr["H2"], 1-x_H2O)

    # set ideal
    atmos.gas_dat["H2O"].eos = phys.EOS_IDEAL
    atmos.gas_dat["H2"].eos = phys.EOS_IDEAL

    # run convective
    println("    con-ide")
    setpt.dry_adiabat!(atmos)
    setpt.saturation!(atmos, "H2O")
    energy.calc_fluxes!(atmos, false, true, true, false)
    atmosphere.calc_observed_rho!(atmos)
    push!(atm_con_ide, deepcopy(atmos))

    # set real gas
    atmos.gas_dat["H2O"].eos = phys.EOS_AQUA
    atmos.gas_dat["H2"].eos = phys.EOS_CMS19

    # run convective
    println("    con-rea")
    setpt.dry_adiabat!(atmos)
    setpt.saturation!(atmos, "H2O")
    energy.calc_fluxes!(atmos, false, true, true, false)
    atmosphere.calc_observed_rho!(atmos)
    push!(atm_con_rea, deepcopy(atmos))

    @printf("--------------------------------- \n")
end

rtol = 0.1
lins = 0
atm_sol_ide = atmosphere.Atmos_t[] # solved, ideal
atm_sol_rea = atmosphere.Atmos_t[] # solved, real
@printf("Running radiative-convective model for 2*%d samples... \n", nsamps)
setpt.stratosphere!(atmos, 500.0)
for i in 1:nsamps
    @printf("    sample %d/%d \n",i,nsamps)

    # update planet
    x_H2O = x_arr[i]
    println("    x_H2O = $x_H2O")
    fill!(atmos.gas_vmr["H2O"], x_H2O)
    fill!(atmos.gas_vmr["H2"], 1-x_H2O)

    # set ideal
    atmos.gas_dat["H2O"].eos = phys.EOS_IDEAL
    atmos.gas_dat["H2"].eos = phys.EOS_IDEAL

    # run RCE
    println("    sol-ide")
    solver.solve_energy!(atmos, save_frames=false, ls_method=lins, conv_rtol=rtol, dx_max=100.0)
    atmosphere.calc_observed_rho!(atmos)
    push!(atm_sol_ide, deepcopy(atmos))

    # set real gas
    atmos.gas_dat["H2O"].eos = phys.EOS_VDW
    atmos.gas_dat["H2"].eos = phys.EOS_CMS19

    # run RCE
    println("    sol-rea")
    solver.solve_energy!(atmos,save_frames=false, ls_method=lins, conv_rtol=rtol, dx_max=50.0)
    atmosphere.calc_observed_rho!(atmos)
    push!(atm_sol_rea, deepcopy(atmos))

    @printf("--------------------------------- \n")
end


println("Done!")

In [ ]:
arr_P = atmos.pl / 1.0e5 # Convert Pa to bar
ylims  = (arr_P[1], arr_P[end])
yticks = 10.0 .^ round.(Int,range( log10(ylims[1]), stop=log10(ylims[2]), step=1))

axl = plot(xlabel="Temperature [K]", legend=:topright, ylabel="Pressure [bar]", legendtitle=L"H$_2$O VMR")
axr = plot(xlabel="Radius [km]")

z0 = 50.0
function r2z(rl)
    return (rl .- radius) ./ 1e3 .+ z0
end

p = Plots.palette(:default)
lw = 3.0
al = 1.0
for i in 1:nsamps
    c = p[i]

    # solved
    _lw = lw
    _al = al
    lbl = @sprintf("%.2f", x_arr[i]*100)

    # real gas
    dat = atm_sol_rea[i]
    plot!(axl, dat.tmpl,    dat.pl/1e5, lc=c, alpha=_al, lw=_lw, label=lbl)
    plot!(axr, r2z(dat.rl), dat.pl/1e5, lc=c, alpha=_al, lw=_lw, label="")

    # ideal gas
    dat = atm_sol_ide[i]
    plot!(axl, dat.tmpl,    dat.pl/1e5, lc=c, alpha=_al, lw=_lw, ls=:dash, label="")
    plot!(axr, r2z(dat.rl), dat.pl/1e5, lc=c, alpha=_al, lw=_lw, ls=:dash, label="")

    # convective
    _lw = lw * 0.5
    _al = al * 0.5

    # real gas
    dat = atm_con_rea[i]
    plot!(axl, dat.tmpl,    dat.pl/1e5, lc=c, alpha=_al, lw=_lw, label="")
    plot!(axr, r2z(dat.rl), dat.pl/1e5, lc=c, alpha=_al, lw=_lw, label="")

    # ideal gas
    dat = atm_con_ide[i]
    plot!(axl, dat.tmpl,    dat.pl/1e5, lc=c, alpha=_al, lw=_lw, ls=:dash, label="")
    plot!(axr, r2z(dat.rl), dat.pl/1e5, lc=c, alpha=_al, lw=_lw, ls=:dash, label="")

end

for ax in (axl, axr)
    yflip!(ax)
    yaxis!(ax, yscale=:log10,ylims=ylims, yticks=yticks)
end
xaxis!(axl, xlims=(0, Inf))
xaxis!(axr, scale=:log10, xlims=(z0, Inf))


fig = plot(axl, axr, size=(1000,700))
display(fig)